# Hierarchical BDH Training on WikiText-103

Train **Hierarchical BDH** on WikiText-103 (~100M tokens, 100x WikiText-2).

## Why WikiText-103?
- **Scale test**: Same domain as WikiText-2 but 100x larger
- **Standard benchmark**: Widely used for LM comparison
- **Long articles**: Full Wikipedia articles, tests long-range modeling

## Expected Outcomes
- Higher perplexity than WikiText-2 (harder, more diverse)
- Better generalization (more data, less overfitting)
- Test hierarchical architecture at scale

## Hardware Requirements
- **Recommended**: A100 40GB+ or equivalent
- **Minimum**: V100 32GB (reduce batch size)
- Training time: ~2-4 hours for 10K steps

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repo
!git clone https://github.com/newsbubbles/bdh.git 2>/dev/null || (cd bdh && git pull)
%cd bdh

In [ ]:
# Install dependencies
!pip install -q torch datasets tqdm matplotlib

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
import math
import json
from datetime import datetime
from tqdm.auto import tqdm

# Import Hierarchical BDH
from bdh_hierarchical import HierarchicalBDH, HierarchicalBDHConfig

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
print(f'PyTorch version: {torch.__version__}')
if device == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name()}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## Load WikiText-103

This is a large dataset (~500MB). First load may take a few minutes.

In [ ]:
from datasets import load_dataset

print('Loading WikiText-103 (this may take a few minutes)...')
dataset = load_dataset('wikitext', 'wikitext-103-raw-v1')

def text_to_bytes(text):
    return torch.tensor(list(text.encode('utf-8')), dtype=torch.long)

# Concatenate all text
print('Processing train split...')
train_text = '\n'.join(dataset['train']['text'])
print('Processing validation split...')
val_text = '\n'.join(dataset['validation']['text'])
print('Processing test split...')
test_text = '\n'.join(dataset['test']['text'])

print('\nConverting to bytes...')
train_data = text_to_bytes(train_text)
val_data = text_to_bytes(val_text)
test_data = text_to_bytes(test_text)

print(f'\nDataset sizes:')
print(f'  Train: {len(train_data):,} bytes ({len(train_data)/1e6:.1f}M)')
print(f'  Val:   {len(val_data):,} bytes ({len(val_data)/1e6:.1f}M)')
print(f'  Test:  {len(test_data):,} bytes ({len(test_data)/1e6:.1f}M)')
print(f'\nTotal: {(len(train_data) + len(val_data) + len(test_data))/1e6:.1f}M bytes')

## Dataset Class

In [ ]:
class ByteDataset(Dataset):
    def __init__(self, data, block_size, patch_size=8):
        assert block_size % patch_size == 0, \
            f'block_size ({block_size}) must be divisible by patch_size ({patch_size})'
        self.data = data
        self.block_size = block_size
    
    def __len__(self):
        return len(self.data) - self.block_size
    
    def __getitem__(self, idx):
        chunk = self.data[idx:idx + self.block_size + 1]
        x = chunk[:-1]
        y = chunk[1:]
        return x, y

## Configuration

For WikiText-103, we use longer training and larger batch sizes.

| Size | Params | Batch | VRAM Est. |
|------|--------|-------|-----------|
| small | 30M | 32 | ~12GB |
| base | 73M | 16 | ~24GB |
| large | 278M | 8 | ~40GB+ |

In [ ]:
# =============================================================
# CHOOSE MODEL SIZE HERE
# =============================================================
MODEL_SIZE = 'base'  # Options: 'tiny', 'small', 'base', 'large'
# =============================================================

# Get model config from preset
preset_map = {
    'tiny': HierarchicalBDHConfig.tiny,
    'small': HierarchicalBDHConfig.small,
    'base': HierarchicalBDHConfig.base,
    'large': HierarchicalBDHConfig.large,
}
model_config = preset_map[MODEL_SIZE](dropout=0.1)  # Less dropout for larger data

# Training config - adjusted for WikiText-103
config = {
    # Model
    'model_size': MODEL_SIZE,
    'patch_size': model_config.patch_size,
    
    # Sequence - longer for WikiText-103
    'block_size': 1024,  # Longer context
    
    # Training - more steps for larger dataset
    'batch_size': {'tiny': 64, 'small': 32, 'base': 16, 'large': 8}[MODEL_SIZE],
    'learning_rate': 3e-4,
    'weight_decay': 0.1,
    'max_steps': 20000,  # More steps for larger dataset
    'warmup_steps': 500,
    
    # Gradient accumulation for effective larger batches
    'gradient_accumulation_steps': 4,
    
    # Validation
    'val_interval': 500,
    'val_batches': 100,
    
    # Early stopping
    'patience': 10,
    
    # Logging
    'log_interval': 100,
}

# Validate
assert config['block_size'] % config['patch_size'] == 0

effective_batch = config['batch_size'] * config['gradient_accumulation_steps']

print(f'Model size: {MODEL_SIZE}')
print(f'Patch size: {config["patch_size"]}')
print(f'Block size: {config["block_size"]} ({config["block_size"] // config["patch_size"]} patches)')
print(f'Batch size: {config["batch_size"]} x {config["gradient_accumulation_steps"]} = {effective_batch} effective')
print()
print('Training config:')
for k, v in config.items():
    print(f'  {k}: {v}')

## Create Model

In [ ]:
# Create model
model = HierarchicalBDH(model_config).to(device)

# Count parameters
params = model.count_parameters()

print(f'\n{"-"*50}')
print(f'HIERARCHICAL BDH - {MODEL_SIZE.upper()}')
print(f'{"-"*50}')
print(f'Global model: {model_config.global_n_layer}L x {model_config.global_n_embd}D x {model_config.global_n_head}H')
print(f'Local model:  {model_config.local_n_layer}L x {model_config.local_n_embd}D x {model_config.local_n_head}H')
print(f'Patch size:   {model_config.patch_size}')
print()
print(f'Parameters:')
print(f'  Global:    {params["global_model"]:>12,}')
print(f'  Local:     {params["local_model"]:>12,}')
print(f'  Patch Emb: {params["patch_embedder"]:>12,}')
print(f'  LM Head:   {params["lm_head"]:>12,}')
print(f'  {"-"*25}')
print(f'  Total:     {params["total"]:>12,} ({params["total"]/1e6:.1f}M)')
print(f'\nGlobal/Local ratio: {params["global_model"]/params["local_model"]:.2f}')

## Setup Training

In [ ]:
# Dataloaders
train_dataset = ByteDataset(train_data, config['block_size'], config['patch_size'])
val_dataset = ByteDataset(val_data, config['block_size'], config['patch_size'])

train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    drop_last=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)

print(f'Train batches: {len(train_loader):,}')
print(f'Val batches: {len(val_loader):,}')
print(f'Steps per epoch: ~{len(train_loader) // config["gradient_accumulation_steps"]:,}')

In [ ]:
# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config['learning_rate'],
    weight_decay=config['weight_decay'],
    betas=(0.9, 0.95),
)

# Learning rate scheduler
def get_lr(step):
    if step < config['warmup_steps']:
        return config['learning_rate'] * step / config['warmup_steps']
    progress = (step - config['warmup_steps']) / (config['max_steps'] - config['warmup_steps'])
    return config['learning_rate'] * 0.5 * (1 + math.cos(math.pi * progress))

# Mixed precision for faster training
scaler = torch.cuda.amp.GradScaler()

print('Optimizer: AdamW with mixed precision (fp16)')
print(f'  lr: {config["learning_rate"]}')
print(f'  weight_decay: {config["weight_decay"]}')
print(f'  warmup_steps: {config["warmup_steps"]}')

## Training Loop

In [ ]:
@torch.no_grad()
def evaluate(model, loader, max_batches=None):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    for i, (x, y) in enumerate(loader):
        if max_batches and i >= max_batches:
            break
        x, y = x.to(device), y.to(device)
        with torch.cuda.amp.autocast():
            _, loss = model(x, y)
        total_loss += loss.item() * y.numel()
        total_tokens += y.numel()
    
    model.train()
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    bpb = avg_loss / math.log(2)
    return avg_loss, perplexity, bpb

In [ ]:
# Training state
history = {
    'step': [],
    'train_loss': [],
    'val_loss': [],
    'val_ppl': [],
    'val_bpb': [],
    'lr': [],
}

best_val_loss = float('inf')
patience_counter = 0
step = 0

# Checkpoints dir
ckpt_dir = Path(f'checkpoints_hierarchical_{MODEL_SIZE}_wikitext103')
ckpt_dir.mkdir(exist_ok=True)

print(f'Checkpoints: {ckpt_dir}')
print('\nStarting training...')
print('=' * 60)

In [ ]:
model.train()
train_iter = iter(train_loader)
running_loss = 0
accum_steps = 0

pbar = tqdm(range(config['max_steps']), desc='Training')

for step in pbar:
    # Accumulate gradients
    optimizer.zero_grad()
    accum_loss = 0
    
    for _ in range(config['gradient_accumulation_steps']):
        try:
            x, y = next(train_iter)
        except StopIteration:
            train_iter = iter(train_loader)
            x, y = next(train_iter)
        
        x, y = x.to(device), y.to(device)
        
        with torch.cuda.amp.autocast():
            _, loss = model(x, y)
            loss = loss / config['gradient_accumulation_steps']
        
        scaler.scale(loss).backward()
        accum_loss += loss.item()
    
    # Update learning rate
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    # Gradient clipping and step
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    scaler.step(optimizer)
    scaler.update()
    
    running_loss += accum_loss
    
    # Logging
    if (step + 1) % config['log_interval'] == 0:
        avg_loss = running_loss / config['log_interval']
        pbar.set_postfix({'loss': f'{avg_loss:.3f}', 'lr': f'{lr:.2e}'})
        running_loss = 0
    
    # Validation
    if (step + 1) % config['val_interval'] == 0:
        val_loss, val_ppl, val_bpb = evaluate(model, val_loader, config['val_batches'])
        train_loss, _, _ = evaluate(model, train_loader, config['val_batches'])
        
        history['step'].append(step + 1)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_ppl'].append(val_ppl)
        history['val_bpb'].append(val_bpb)
        history['lr'].append(lr)
        
        gap = val_loss - train_loss
        print(f'\nStep {step+1}: train={train_loss:.3f}, val={val_loss:.3f}, ppl={val_ppl:.2f}, bpb={val_bpb:.3f}, gap={gap:.3f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save({
                'step': step + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'model_config': model_config.__dict__,
                'val_loss': val_loss,
                'val_ppl': val_ppl,
                'val_bpb': val_bpb,
                'dataset': 'wikitext-103',
            }, ckpt_dir / 'best.pt')
            print(f'  ✓ New best! Saved to {ckpt_dir}/best.pt')
        else:
            patience_counter += 1
            print(f'  No improvement ({patience_counter}/{config["patience"]})')
        
        if patience_counter >= config['patience']:
            print(f'\n⚠️ Early stopping at step {step+1}')
            break

print('\n' + '=' * 60)
print('Training complete!')
print(f'Best val loss: {best_val_loss:.4f}')

## Save Results

In [ ]:
# Save training history
with open(ckpt_dir / 'training_history.json', 'w') as f:
    json.dump(history, f, indent=2)

# Save config
full_config = {
    **config,
    'model_config': model_config.__dict__,
    'parameters': params,
    'dataset': 'wikitext-103',
    'dataset_size_bytes': len(train_data),
}
with open(ckpt_dir / 'config.json', 'w') as f:
    json.dump(full_config, f, indent=2)

print(f'Saved history and config to {ckpt_dir}')

## Plot Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 4, figsize=(18, 4))

# Loss curves
ax = axes[0]
ax.plot(history['step'], history['train_loss'], 'b-', label='Train', linewidth=2)
ax.plot(history['step'], history['val_loss'], 'r-', label='Val', linewidth=2)
ax.set_xlabel('Step')
ax.set_ylabel('Loss')
ax.set_title('Loss Curves')
ax.legend()
ax.grid(True, alpha=0.3)

# Perplexity
ax = axes[1]
ax.plot(history['step'], history['val_ppl'], 'g-', linewidth=2)
ax.set_xlabel('Step')
ax.set_ylabel('Perplexity')
ax.set_title('Validation Perplexity')
ax.grid(True, alpha=0.3)

# Bits per byte
ax = axes[2]
ax.plot(history['step'], history['val_bpb'], 'm-', linewidth=2)
ax.set_xlabel('Step')
ax.set_ylabel('BPB')
ax.set_title('Bits Per Byte')
ax.grid(True, alpha=0.3)

# Gap
ax = axes[3]
gaps = [v - t for v, t in zip(history['val_loss'], history['train_loss'])]
ax.fill_between(history['step'], gaps, alpha=0.5, color='orange')
ax.plot(history['step'], gaps, 'orange', linewidth=2)
ax.axhline(y=0.5, color='red', linestyle='--', label='Overfitting threshold')
ax.set_xlabel('Step')
ax.set_ylabel('Val - Train Loss')
ax.set_title('Overfitting Gap')
ax.legend()
ax.grid(True, alpha=0.3)

plt.suptitle(f'Hierarchical BDH ({MODEL_SIZE}) - WikiText-103', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(ckpt_dir / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## Evaluate Best Model

In [ ]:
# Load best checkpoint
ckpt = torch.load(ckpt_dir / 'best.pt')
model.load_state_dict(ckpt['model_state_dict'])

# Full validation
print('Evaluating best model on full validation set...')
val_loss, val_ppl, val_bpb = evaluate(model, val_loader)
print(f'Val Loss: {val_loss:.4f}')
print(f'Val Perplexity: {val_ppl:.2f}')
print(f'Val BPB: {val_bpb:.3f}')

# Test set
test_dataset = ByteDataset(test_data, config['block_size'], config['patch_size'])
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=2)

print('\nEvaluating on test set...')
test_loss, test_ppl, test_bpb = evaluate(model, test_loader)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Perplexity: {test_ppl:.2f}')
print(f'Test BPB: {test_bpb:.3f}')

## Test Generation

In [ ]:
def generate_text(model, prompt, max_new_tokens=200, temperature=0.8, top_k=50):
    model.eval()
    prompt_bytes = list(prompt.encode('utf-8'))
    idx = torch.tensor([prompt_bytes], device=device, dtype=torch.long)
    
    with torch.no_grad():
        output = model.generate(idx, max_new_tokens=max_new_tokens, temperature=temperature, top_k=top_k)
    
    output_bytes = output[0].tolist()
    try:
        text = bytes(output_bytes).decode('utf-8', errors='replace')
    except:
        text = ''.join(chr(b) if 32 <= b < 127 else '?' for b in output_bytes)
    return text

prompts = [
    'The history of',
    'In 1920, the',
    'Scientists discovered',
    'The capital of France',
]

print('=' * 60)
print('GENERATION SAMPLES')
print('=' * 60)

for prompt in prompts:
    print(f'\nPrompt: "{prompt}"')
    print('-' * 40)
    output = generate_text(model, prompt, max_new_tokens=150, temperature=0.8)
    print(output)
    print()

## Summary

In [ ]:
print('=' * 60)
print('WIKITEXT-103 RESULTS SUMMARY')
print('=' * 60)
print()
print(f'Model: Hierarchical BDH ({MODEL_SIZE})')
print(f'Parameters: {params["total"]:,} ({params["total"]/1e6:.1f}M)')
print(f'Dataset: WikiText-103 ({len(train_data)/1e6:.1f}M bytes)')
print()
print('Final Metrics:')
print(f'  Test Loss:      {test_loss:.4f}')
print(f'  Test Perplexity: {test_ppl:.2f}')
print(f'  Test BPB:       {test_bpb:.3f}')
print()
print('Comparison (expected ranges):')
print('  WikiText-2 Hierarchical: PPL ~1.2 (smaller, easier)')
print('  WikiText-103 should show higher PPL due to diversity')
print('  But better generalization (larger training set)')

## Download Checkpoint

In [ ]:
import subprocess

zip_name = f'hierarchical_bdh_{MODEL_SIZE}_wikitext103.zip'
subprocess.run(['zip', '-r', zip_name, str(ckpt_dir)], check=True)
print(f'Created: {zip_name}')

try:
    from google.colab import files
    print('Downloading checkpoint...')
    files.download(zip_name)
    print('\n✅ Download started!')
except ImportError:
    print(f'Checkpoints saved to: {ckpt_dir}')
    print(f'Zip file: {zip_name}')